In [1]:
import pandas as pd
import numpy as np
import json
import os
import capstone_json_functions as cjf

In [2]:
directory = "data/kickstarter_json/"
filename = "2022-06-09.json"
file_path = f"{directory}{filename}"

In [19]:
with open(file_path) as fp:
    lines = fp.readlines()


In [45]:
df = pd.DataFrame()
category_dict = dict()
for line in lines:
    line_dict = json.loads(line[line.index('{"id":'):-2])
    category_dict[line_dict["category"]["id"]] = (line_dict["category"]["name"],line_dict["category"]["analytics_name"],line_dict["category"]["slug"])
    #print(line_dict["category"])
    #print(category_dict)
    # df_temp = pd.json_normalize(line_dict["data"],sep='_')
    # if df_temp["data_category_id"]
    # df = pd.concat([df,df_temp])
    # break

In [56]:
list_of_categories = []
for key, value in category_dict.items():
    list_of_categories.append([key,value])
    #print(f"id: {key} ---> {value}")
list_of_categories.sort()
with open("kickstarter_categories.txt","w") as fp:
    for item in list_of_categories:
        #print(f"id: {item[0]} ---> {item[1]}")
        fp.writelines(f"id: {item[0]} ---> {item[1]}\n")

In [ ]:
df_test.head()

In [ ]:
df_temp = cjf.create_json_df(file_path)

In [ ]:
df2 = cjf.df_from_json(df_temp)

In [ ]:
df2[["currency","currency_trailing_code","static_usd_rate",
            "usd_pledged",
            "converted_pledged_amount","slug","name"]]

In [ ]:
df2.head()

In [ ]:
df_without_profile = df2.loc[:,df2.columns.str.contains("category")]

In [ ]:
df_without_profile.drop_duplicates()

In [ ]:
df = pd.DataFrame()

for idx,row in json_df.iterrows():
    keys_to_pop = []
    this_row = dict(row["data"])
    for key in this_row.keys():
        if key not in keys_to_keep:
            keys_to_pop.append(key)
    for trash_keys in keys_to_pop:
        this_row.pop(trash_keys)
    #print(this_row)
    if dict(this_row["category"])["id"] == 34:
        df_temp = pd.json_normalize(this_row,sep='_')
        df = pd.concat([df,df_temp])
    #print(df_temp.info())


In [ ]:
data = [json.loads(line)
        for line in open(file_path)]

In [ ]:
data_normalized = pd.json_normalize(data,record_path='data',sep='_',record_prefix='data_')

In [ ]:
data_df = pd.DataFrame()
for entry in data:
    data_dict = entry.copy()
    df = pd.DataFrame(data_dict)
    df.drop(["table_id",'run_id','robot_id'],axis=1,inplace=True)
    df = df.T
    if ": 34," in df["category"].to_string():
        data_df = pd.concat([data_df,df])
    else:
        continue

In [ ]:
df = pd.DataFrame()
for line in open(file_path):
    data_temp = json.loads(line)
    df_temp = pd.json_normalize(data_temp,sep='_')
    df = pd.concat([df,df_temp])
